In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/train.csv
/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/test.csv
/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/sample_submit.csv


In [2]:
train=pd.read_csv("/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/train.csv")
test=pd.read_csv("/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/test.csv")
submit=pd.read_csv("/kaggle/input/2021-ai-w10-p1/2021-ai-w10-p1/sample_submit.csv")

In [3]:
x=train.drop("Category",axis=1)
y=train["Category"]

x=np.array(x)
y=np.array(y)
test=np.array(test)

In [4]:
import torch
   
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
class MNIST_Dataset(torch.utils.data.Dataset):
  def __init__(self,split,transform=None):
    super(MNIST_Dataset,self).__init__()

    self.split=split.upper()
    self.transform = transform

    assert self.split in {"TRAIN","TEST"}

    if self.split=='TRAIN':
      self.images=x
      self.labels=y
    
    elif self.split=='TEST':
      self.images=test
  def __getitem__(self,index):
    image=torch.FloatTensor(self.images[index])
    image=image.reshape(-1,28,28)

    if self.split=='TRAIN':
      label=self.labels[index]
      return image,label

    elif self.split=='TEST':
      return image

    label=self.labels[index]

  def __len__(self):
    return len(self.images)

In [6]:

batch_size=128
workers=2
learning_rate=0.001
total_epoch=20
n_classes=10

TRAIN_dataset=MNIST_Dataset(split='train')
TRAIN_Loader=torch.utils.data.DataLoader(dataset=TRAIN_dataset,batch_size=batch_size,shuffle=True,num_workers=workers)

TEST_dataset=MNIST_Dataset(split='test')
TEST_Loader=torch.utils.data.DataLoader(dataset=TEST_dataset,batch_size=batch_size,shuffle=False,num_workers=workers)

In [7]:
class CNN(torch.nn.Module):
  def __init__(self,n_classes):
    super(CNN,self).__init__()

    self.conv1=torch.nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1) #1채널의 필터가 32개 만들어진 것
    self.bn1=torch.nn.BatchNorm2d(32)

    self.conv2=torch.nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1) 
    self.bn2=torch.nn.BatchNorm2d(64)

    self.conv3=torch.nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1)
    self.bn3=torch.nn.BatchNorm2d(128)

    self.pooling=torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc1=torch.nn.Linear(in_features=3*3*128,out_features=625,bias=True)
    self.fc2=torch.nn.Linear(in_features=625,out_features=n_classes,bias=True)

    self.init_weight()


  def forward(self,x): # x-> image data [128,1,28,28]
    out=self.pooling(torch.nn.functional.relu(self.bn1(self.conv1(x))))
    out=self.pooling(torch.nn.functional.relu(self.bn2(self.conv2(out))))
    out=self.pooling(torch.nn.functional.relu(self.bn3(self.conv3(out))))

    out=out.view(-1,3*3*128) #out.shape =[128,3*3*128]

    out=torch.nn.functional.relu(self.fc1(out))
    out=self.fc2(out) #out.shape [128,10] 

    return out
  
  def init_weight(self):
    for m in self.children():
      if isinstance(m,torch.nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
        torch.nn.init.constant_(m.bias,0)
        '''
      elif isinstance(m,torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
        torch.nn.init.constant_(m.bias,0)
      elif isinstance(m,torch.nn.BatchNorm2d):
        torch.nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
        torch.nn.init.constant_(m.bias,0)
        '''
    print("weight init")

In [8]:
Network=CNN(n_classes=n_classes)
Network=Network.to(device)

weight init


In [9]:
Loss=torch.nn.CrossEntropyLoss() #분류 문제를 해결하기 위한 Cross Entropy Loss
Loss=Loss.to(device)

optimizer=torch.optim.Adam(Network.parameters(),lr=learning_rate) #최적화를 위한 Optimizer

In [10]:
Network.train()
for epoch in range(total_epoch):
  correct_pred = 0
  epoch_loss = 0
  for X,Y in TRAIN_Loader:
    X= X.to(device)
    Y=torch.LongTensor(Y).to(device)

    optimizer.zero_grad() #하나의 배치에 대해서 Gradient업데이트하기 전에 Gradient 초기화를 담당
    hypothesis=Network(X) #Feed forward 과정이 진행
    _, preds=torch.max(hypothesis,1)
    Cost=Loss(hypothesis,Y)
    Cost.backward() #backpropagation
    optimizer.step() #parameter update

    correct_pred = correct_pred + torch.sum(preds==Y)
    epoch_loss = epoch_loss + Cost.item()*X.size(0)

  print("epoch : {:02d} , Accuracy : {:.4f} , Loss : {:.4f}".format(epoch+1 , correct_pred/len(TRAIN_Loader.dataset),epoch_loss/len(TRAIN_Loader.dataset)))

epoch : 01 , Accuracy : 0.9614 , Loss : 0.1287
epoch : 02 , Accuracy : 0.9881 , Loss : 0.0377
epoch : 03 , Accuracy : 0.9908 , Loss : 0.0281
epoch : 04 , Accuracy : 0.9927 , Loss : 0.0227
epoch : 05 , Accuracy : 0.9936 , Loss : 0.0199
epoch : 06 , Accuracy : 0.9951 , Loss : 0.0148
epoch : 07 , Accuracy : 0.9959 , Loss : 0.0121
epoch : 08 , Accuracy : 0.9973 , Loss : 0.0086
epoch : 09 , Accuracy : 0.9969 , Loss : 0.0094
epoch : 10 , Accuracy : 0.9969 , Loss : 0.0088
epoch : 11 , Accuracy : 0.9970 , Loss : 0.0090
epoch : 12 , Accuracy : 0.9974 , Loss : 0.0072
epoch : 13 , Accuracy : 0.9983 , Loss : 0.0056
epoch : 14 , Accuracy : 0.9980 , Loss : 0.0058
epoch : 15 , Accuracy : 0.9980 , Loss : 0.0064
epoch : 16 , Accuracy : 0.9984 , Loss : 0.0048
epoch : 17 , Accuracy : 0.9987 , Loss : 0.0036
epoch : 18 , Accuracy : 0.9978 , Loss : 0.0067
epoch : 19 , Accuracy : 0.9984 , Loss : 0.0045
epoch : 20 , Accuracy : 0.9990 , Loss : 0.0028


In [11]:
prediction=list()
with torch.no_grad() : #test시에ㅡㄴ작성
  for X in TEST_Loader:
    X=X.to(device)
    hypothesis=Network(X)
    _,preds=torch.max(hypothesis,1)
    prediction.extend(preds.tolist())

In [12]:
submit['Category']=prediction
submit.to_csv('submit.csv',index=False)